In [ ]:
import pickle 

with open('user_embedding.pkl', 'rb') as f:
    user_embedding = pickle.load(f)

with open('event_embeddings.pkl', 'rb') as f:
    event_embeddings = pickle.load(f)


In [ ]:
import pymysql
import pandas as pd
import os
import openai

mysql_pass = os.getenv("MYSQL_PASSWORD")
connection = pymysql.connect(
    host='localhost',
    user='root',
    password=mysql_pass,
    db='mysfevents'
)

# Execute a SQL query to retrieve data from a table
query = 'SELECT event_id FROM liked_events'
data = pd.read_sql_query(query, connection)

second_query = 'SELECT id FROM events'
data2 = pd.read_sql_query(second_query, connection)

# Close the database connection
connection.close()

# Print the dataframe
liked_event_ids = data['event_id'].to_list()
event_ids = data2['id'].to_list()


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

liked_events_mask = np.isin(event_ids, liked_event_ids)
masked_event_embeddings = np.copy(event_embeddings)
masked_event_embeddings[liked_events_mask] = 0.0
user_embedding_reshaped = np.reshape(user_embedding, (1, -1))

all_embeddings = np.concatenate([user_embedding_reshaped, masked_event_embeddings])

similarity_scores = cosine_similarity(all_embeddings)[0][1:]

In [ ]:
import pymysql

# create a connection to your MySQL database
connection = pymysql.connect(
    host='localhost',
    user='root',
    password=mysql_pass,
    db='mysfevents'
)

# get a cursor to execute SQL statements
cursor = connection.cursor()

# update the similarity scores in the MySQL table
for i, event_id in enumerate(event_ids):
    similarity = similarity_scores[i]
    sql = "UPDATE events SET similarity = %s WHERE id = %s"
    val = (similarity, event_id)
    cursor.execute(sql, val)

# commit the changes to the database
connection.commit()

# close the database connection
cursor.close()
connection.close()
